In [17]:
import matplotlib.pyplot as plt
from funcs import *
import os

os.chdir(R'C:\Users\adria\Desktop\STUDIA_FOLDERY\TELEDETEKCJA')

In [18]:
cir_raster = r"TIFs\CIR_2015.tif"
cir_dataset = read_spatial_raster(cir_raster)

rgb_raster = r"TIFs\RGB_2015.tif"
rgb_dataset = read_spatial_raster(rgb_raster)

c:\Users\adria\anaconda3\envs\teledetekcja\Lib\site-packages\osgeo\gdal.py:8817: RuntimeWarning: CIR_2015.tif: JPEGFixupTagsSubsamplingSec:Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [1,1]
  return _gdal.Open(*args)
c:\Users\adria\anaconda3\envs\teledetekcja\Lib\site-packages\osgeo\gdal.py:8817: RuntimeWarning: RGB_2015.tif: JPEGFixupTagsSubsamplingSec:Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [1,1]
  return _gdal.Open(*args)


In [19]:
band_nir = cir_dataset.GetRasterBand(1)
array_nir = band_nir.ReadAsArray()
array_nir = np.copy(array_nir)

band_red = cir_dataset.GetRasterBand(2)
array_red = band_red.ReadAsArray()
array_red = np.copy(array_red)

band_green = cir_dataset.GetRasterBand(3)
array_green = band_green.ReadAsArray()
array_green = np.copy(array_green)

band_blue = rgb_dataset.GetRasterBand(3)
array_blue = band_blue.ReadAsArray()
array_blue = np.copy(array_blue)

In [20]:
ids = np.array([7, 0, 3])

vector_features = gpd.read_file(R"budynki\budynki.shp")

features = reproject_geodataframe(vector_features, cir_dataset.GetProjection())
features = convert_to_pixel_system(features, cir_dataset.GetGeoTransform())

In [ ]:
for id in ids:
    example_feature = features.iloc[id]
    example_feature = example_feature['geometry']

    bounds = example_feature.bounds
    bounds = np.float64(bounds)

    bounds[:2] = np.floor(bounds[:2])
    bounds[2:] = np.ceil(bounds[2:])
    bounds = np.int64(bounds)

    fragment_nir = array_nir[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_red = array_red[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_green = array_green[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_blue = array_blue[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    # plt.imsave(f"img\\nir_budynek{id}.png", fragment_nir, cmap='gray')
    # plt.imsave(f"img\\red_budynek{id}.png", fragment_red, cmap='gray')
    # plt.imsave(f"img\\green_budynek{id}.png", fragment_green, cmap='gray')
    # plt.imsave(f"img\\blue_budynek{id}.png", fragment_blue, cmap='gray')

    feature_in_fragment_frame = shapely.affinity.translate(example_feature, -bounds[0], -bounds[1])

    from rasterio.features import rasterize

    # maska
    no_data_mask = rasterize([feature_in_fragment_frame], fragment_nir.shape)
    no_data_mask = np.bool_(no_data_mask)
    no_data_mask = ~no_data_mask  
    
    # nir
    masked_fragment = np.copy(fragment_nir)
    masked_fragment[no_data_mask] = 0

    pixel_values = fragment_nir[~no_data_mask]  # Select just valid pixels
    print(pixel_values.min(), pixel_values.max(), pixel_values.mean())

    plt.imsave(f"img\\nir_budynek{id}.png", masked_fragment, cmap='gray')

    # red
    masked_fragment = np.copy(fragment_red)
    masked_fragment[no_data_mask] = 0

    pixel_values = fragment_red[~no_data_mask]
    print(pixel_values.min(), pixel_values.max(), pixel_values.mean())

    plt.imsave(f"img\\red_budynek{id}.png", masked_fragment, cmap='gray')

    # green
    masked_fragment = np.copy(fragment_green)
    masked_fragment[no_data_mask] = 0

    pixel_values = fragment_green[~no_data_mask]
    print(pixel_values.min(), pixel_values.max(), pixel_values.mean())

    plt.imsave(f"img\\green_budynek{id}.png", masked_fragment, cmap='gray')

    # blue
    masked_fragment = np.copy(fragment_blue)
    masked_fragment[no_data_mask] = 0

    pixel_values = fragment_blue[~no_data_mask] 
    print(pixel_values.min(), pixel_values.max(), pixel_values.mean())

    plt.imsave(f"img\\blue_budynek{id}.png", masked_fragment, cmap='gray')

    # rgb
    rgb = np.dstack([fragment_red, fragment_green, fragment_blue])
    rgb[no_data_mask] = 0

    plt.imsave(f"img\\rgb_budynek{id}.png", rgb, cmap='gray')

    # irgb
    irgb = np.dstack([fragment_nir, fragment_green, fragment_blue])
    irgb[no_data_mask] = 0

    plt.imsave(f"img\\irgb_budynek{id}.png", irgb, cmap='gray')

[113  87  87 ... 151 143 150] 48 157 77.2828999211978
40 131 70.44365642237983
43 134 71.22695035460993
37 120 65.39322301024428
[ 86  77  90 ... 125 121 108] 39 187 101.36163265306122
47 203 127.01306122448979
60 217 132.78448979591838
43 205 128.09142857142857
[126 116 129 ... 101 111 114] 59 178 90.41953488372093
60 189 98.90325581395349
55 193 96.62139534883721
44 191 92.05953488372093
